<a href="https://colab.research.google.com/github/10evoy/university/blob/main/test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# --- 1. Генерация датасета ---
def generate_dataset(num_samples=5000, img_size=28):
    """Генерирует изображения кругов, квадратов и треугольников."""
    X = np.zeros((num_samples, img_size, img_size))
    y = np.zeros(num_samples, dtype=np.uint8)

    for i in range(num_samples):
        shape_type = np.random.randint(0, 3)
        y[i] = shape_type

        if shape_type == 0:  # Круг
            center = np.random.randint(10, img_size - 10, size=2)
            radius = np.random.randint(5, 10)
            for x in range(img_size):
                for y_coord in range(img_size):
                    if (x - center[0])**2 + (y_coord - center[1])**2 <= radius**2:
                        X[i, x, y_coord] = 1

        elif shape_type == 1:  # Квадрат
            start_x = np.random.randint(5, img_size - 15)
            start_y = np.random.randint(5, img_size - 15)
            size = np.random.randint(8, 15)
            X[i, start_x:start_x+size, start_y:start_y+size] = 1

        else:  # Треугольник
            p1 = np.random.randint(5, img_size - 5, size=2)
            p2 = np.random.randint(5, img_size - 5, size=2)
            p3 = np.random.randint(5, img_size - 5, size=2)

            # Простой способ закрасить треугольник
            for x in range(img_size):
                for y_coord in range(img_size):
                    # Используем барицентрические координаты (упрощенный вариант)
                    # Это не идеальный треугольник, но для задачи подходит
                    if ( (y_coord - p1[1]) * (p2[0] - p1[0]) - (x - p1[0]) * (p2[1] - p1[1]) ) * \
                       ( (y_coord - p1[1]) * (p3[0] - p1[0]) - (x - p1[0]) * (p3[1] - p1[1]) ) < 0 and \
                       ( (y_coord - p2[1]) * (p1[0] - p2[0]) - (x - p2[0]) * (p1[1] - p2[1]) ) * \
                       ( (y_coord - p2[1]) * (p3[0] - p2[0]) - (x - p2[0]) * (p3[1] - p2[1]) ) < 0:
                        X[i, x, y_coord] = 1

    return X, y

# --- 2. Подготовка данных ---
print("1. Генерация и подготовка данных...")
X, y = generate_dataset(num_samples=5000, img_size=28)

# Нормализация
X = X.astype('float32') / 255.0

# One-hot кодирование меток
y_categorical = to_categorical(y, num_classes=3)

# Разделение на обучающую и тестовую выборки (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)

# Добавление размерности канала для CNN
X_train_cnn = X_train[..., np.newaxis]
X_test_cnn = X_test[..., np.newaxis]

# --- 3. Создание моделей (Задание 10) ---
print("\n2. Создание моделей...")

# а) Полносвязная сеть (FCN)
model_fcn = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

# б) Сверточная сеть (CNN)
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

# --- 4. Обучение и сравнение моделей (Задание 11) ---
def train_and_evaluate(model, X_tr, y_tr, X_te, y_te, model_name):
    """Обучает модель и возвращает время, точность и количество параметров."""
    print(f"\n--- Обучение модели: {model_name} ---")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=0, validation_split=0.1)
    end_time = time.time()

    training_time = end_time - start_time

    loss, accuracy = model.evaluate(X_te, y_te, verbose=0)

    num_params = model.count_params()

    print(f"Время обучения: {training_time:.2f} сек.")
    print(f"Точность на тестовой выборке: {accuracy:.4f}")
    print(f"Количество параметров: {num_params}")

    return {
        "name": model_name,
        "time": training_time,
        "accuracy": accuracy,
        "params": num_params
    }

# Обучаем и оцениваем обе модели
results_fcn = train_and_evaluate(model_fcn, X_train, y_train, X_test, y_test, "Полносвязная сеть (FCN)")
results_cnn = train_and_evaluate(model_cnn, X_train_cnn, y_train, X_test_cnn, y_test, "Сверточная сеть (CNN)")

# --- 5. Анализ результатов ---
print("\n--- Сравнительный анализ производительности ---")
print(f"{'Модель':<30} {'Точность':<12} {'Время (с)':<12} {'Параметры':<12}")
print("-" * 66)
print(f"{results_fcn['name']:<30} {results_fcn['accuracy']:<12.4f} {results_fcn['time']:<12.2f} {results_fcn['params']:<12}")
print(f"{results_cnn['name']:<30} {results_cnn['accuracy']:<12.4f} {results_cnn['time']:<12.2f} {results_cnn['params']:<12}")

"""
Анализ результатов:
- Ожидается, что CNN покажет более высокую точность, так как сверточные слои лучше
  улавливают пространственные признаки фигур (границы, углы).
- Время обучения CNN, скорее всего, будет больше из-за более сложных вычислений (свертки).
- Количество параметров у CNN также будет выше, особенно из-за полносвязного слоя
  после выравнивания большой карты признаков.
- FCN, несмотря на простоту, может показать неплохой результат на такой простой задаче,
  но будет менее устойчива к смещению/масштабированию фигур.
"""



1. Генерация и подготовка данных...

2. Создание моделей...

--- Обучение модели: Полносвязная сеть (FCN) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Время обучения: 6.14 сек.
Точность на тестовой выборке: 0.7740
Количество параметров: 100867

--- Обучение модели: Сверточная сеть (CNN) ---
Время обучения: 36.47 сек.
Точность на тестовой выборке: 0.8780
Количество параметров: 1010563

--- Сравнительный анализ производительности ---
Модель                         Точность     Время (с)    Параметры   
------------------------------------------------------------------
Полносвязная сеть (FCN)        0.7740       6.14         100867      
Сверточная сеть (CNN)          0.8780       36.47        1010563     


'\nАнализ результатов:\n- Ожидается, что CNN покажет более высокую точность, так как сверточные слои лучше\n  улавливают пространственные признаки фигур (границы, углы).\n- Время обучения CNN, скорее всего, будет больше из-за более сложных вычислений (свертки).\n- Количество параметров у CNN также будет выше, особенно из-за полносвязного слоя\n  после выравнивания большой карты признаков.\n- FCN, несмотря на простоту, может показать неплохой результат на такой простой задаче,\n  но будет менее устойчива к смещению/масштабированию фигур.\n'

In [11]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical

# --- 1. Загрузка и подготовка данных MNIST ---
print("1. Загрузка и подготовка данных MNIST...")
(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

subset_size = 1000
x_train_full = x_train_full[:subset_size]
y_train_full = y_train_full[:subset_size]

# Нормализация
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot кодирование
y_train_full = to_categorical(y_train_full, 10)
y_test = to_categorical(y_test, 10)

# Разделение обучающей выборки на обучающую и валидационную (90%/10%)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, test_size=0.1, random_state=42
)

# Добавление размерности канала
x_train = x_train[..., np.newaxis]
x_val = x_val[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# --- 2. Определение сетки гиперпараметров для эксперимента (Задание 12) ---
param_grid = {
    'filters_1': [16, 32],
    'filters_2': [32, 64],
    'kernel_size': [(3, 3)],
    'dense_neurons': [64, 128, 256],
    'dropout_rate': [0.3, 0.5]
}

all_results = []

# --- 3. Цикл экспериментов (Задание 13) ---
print("\n2. Начало экспериментов по оптимизации гиперпараметров...")
total_combinations = len(param_grid['filters_1']) * len(param_grid['filters_2']) * \
                     len(param_grid['kernel_size']) * len(param_grid['dense_neurons']) * \
                     len(param_grid['dropout_rate'])

experiment_count = 0

for f1 in param_grid['filters_1']:
    for f2 in param_grid['filters_2']:
        for k_size in param_grid['kernel_size']:
            for d_neurons in param_grid['dense_neurons']:
                for d_rate in param_grid['dropout_rate']:
                    experiment_count += 1
                    print(f"\n--- Эксперимент {experiment_count}/{total_combinations} ---")
                    print(f"Параметры: F1={f1}, F2={f2}, Kernel={k_size}, Dense={d_neurons}, Dropout={d_rate}")

                    # Создание модели с текущими гиперпараметрами
                    model = Sequential([
                        Conv2D(f1, k_size, activation='relu', input_shape=(28, 28, 1)),
                        MaxPooling2D((2, 2)),
                        Conv2D(f2, k_size, activation='relu'),
                        MaxPooling2D((2, 2)),
                        Flatten(),
                        Dense(d_neurons, activation='relu'),
                        Dropout(d_rate),
                        Dense(10, activation='softmax')
                    ])

                    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

                    # Обучение на 5 эпохах
                    history = model.fit(
                        x_train, y_train,
                        epochs=5,
                        batch_size=128,
                        verbose=0, # Отключаем вывод для чистоты лога
                        validation_data=(x_val, y_val)
                    )

                    # Запись точности на валидационной выборке
                    val_accuracy = history.history['val_accuracy'][-1]
                    print(f"Точность на валидации: {val_accuracy:.4f}")

                    all_results.append({
                        'val_accuracy': val_accuracy,
                        'params': {
                            'filters_1': f1,
                            'filters_2': f2,
                            'kernel_size': k_size,
                            'dense_neurons': d_neurons,
                            'dropout_rate': d_rate
                        }
                    })

# --- 4. Анализ и определение оптимальной конфигурации (Задание 14) ---
print("\n\n--- Анализ результатов эксперимента ---")

# Сортировка результатов по точности
all_results_sorted = sorted(all_results, key=lambda x: x['val_accuracy'], reverse=True)

# Вывод топ-5 результатов
print("\nТоп-5 лучших конфигураций:")
for i, res in enumerate(all_results_sorted[:5]):
    print(f"{i+1}. Точность: {res['val_accuracy']:.4f}, Параметры: {res['params']}")

# Определение лучшей конфигурации
best_result = all_results_sorted[0]
print("\n--- ОПТИМАЛЬНАЯ КОНФИГУРАЦИЯ ---")
print(f"Лучшая точность на валидации: {best_result['val_accuracy']:.4f}")
print("Оптимальные гиперпараметры:")
for key, value in best_result['params'].items():
    print(f"  - {key}: {value}")

"""
Построение полного графика зависимости от всех комбинаций было бы неинформативным
из-за большого количества измерений (многомерный график).
Поэтому наиболее наглядным и полезным является таблица с отсортированными
результатами, которая позволяет легко определить лучшую комбинацию.
"""


1. Загрузка и подготовка данных MNIST...

2. Начало экспериментов по оптимизации гиперпараметров...

--- Эксперимент 1/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=64, Dropout=0.3
Точность на валидации: 0.8200

--- Эксперимент 2/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=64, Dropout=0.5
Точность на валидации: 0.7600

--- Эксперимент 3/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=128, Dropout=0.3
Точность на валидации: 0.8100

--- Эксперимент 4/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=128, Dropout=0.5
Точность на валидации: 0.8300

--- Эксперимент 5/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=256, Dropout=0.3
Точность на валидации: 0.8600

--- Эксперимент 6/24 ---
Параметры: F1=16, F2=32, Kernel=(3, 3), Dense=256, Dropout=0.5
Точность на валидации: 0.7800

--- Эксперимент 7/24 ---
Параметры: F1=16, F2=64, Kernel=(3, 3), Dense=64, Dropout=0.3
Точность на валидации: 0.8000

--- Эксперимент 8/24 ---
Параметры: F1=16, F2=64, Kernel=(3, 3), Dens

'\nПостроение полного графика зависимости от всех комбинаций было бы неинформативным\nиз-за большого количества измерений (многомерный график).\nПоэтому наиболее наглядным и полезным является таблица с отсортированными\nрезультатами, которая позволяет легко определить лучшую комбинацию.\n'